In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
#######
import statistics as stat
import scipy as sp
from bisect import bisect  # to compute time index
import pygsp
from pygsp import graphs
from pygsp import filters
#######
from matrixprofile.matrixProfile import stomp
from matrixprofile.motifs import motifs
from matrixprofile.discords import discords

In [2]:
from anx import ModelUtils
from anx import Driver

In [3]:
os.chdir("/home/manuelherrera/Working/Data/anxTest")

retval = os.getcwd()
print("Directory successfully changed", retval) # now we can load data and modules at the new working folder

Directory successfully changed /home/manuelherrera/Working/Data/anxTest


The idea is to try GFT TS and GFT-Vertex TS and compare results of anomaly detection given a node failure using the network simulator

In [4]:
#G1 = nx.erdos_renyi_graph(13,0.5)
  
#plt.figure(figsize = (12, 12)) 
#nx.draw_networkx(G1)

In [43]:
G1 = nx.barabasi_albert_graph(92,1)
#plt.figure(figsize = (12, 12)) 
#nx.draw_networkx(G2)

In [44]:
# remember the default path is settled as "/home/manuelherrera/Working/Data/anxTest/"

# To write manually "GSP_test1.param" (done in path folder)

fp = open("GSP_test1.topo", "w")
for edge in G1.edges():
    fp.write("{:d} {:d}\n".format(edge[0], edge[1])) # :s for string names 
fp.close()

fp = open("GSP_test1.type", "w")
for node in range(len(G1.nodes())+1):
    fp.write("{:d} n\n".format(node))
fp.close()

In [45]:
links = ModelUtils.load_links("GSP_test1")
node_types = ModelUtils.load_node_types("GSP_test1")
param = ModelUtils.load_param("GSP_test1")
M = ModelUtils.make_model(links, node_types, param, "GSP_test1")

# Folder needs to previously exist 
# remember the default path is settled as "/home/manuelherrera/Working/Data/anxTest/"
n_sim = 720
Driver.run_sim(M, t=n_sim, output_dir="output_GSP_Test") 

 [+] Verifying network topology..All good
 [+] Total number of links: 91
 [+] Unique links: 91
 [+] Loading node types
 [+] Loading network parameters
 [+] Preparing model
 [+] Checking model input..All good
 [*] Calculating shortest paths beween all nodes
 [+] Initialising the simulation...
 [+] Total simulation time is 720.00 time units
 [+] Found 92 nodes and 91 links
 [+] Simulations started...

 [ Running progress bar ]
 [########################################] 99%

 [+] Simulation completed
 [+] Printing summary statistics:

	Total packets sent: 1,262,007
	Total packets recv: 732,618

 [+] Saving queue monitors..
 [+] Saving generated packets..
 [+] Saving forwarded packets..
 [+] Saving received packets..
 [+] Saving discarded packets..


In [46]:
G = G1.copy()

In [47]:
traffic_sim = []
p = "/home/manuelherrera/Working/Data/anxTest/output_GSP_Test/"
for j in range(n_sim): # to move during the n_sim time units
    for i in G.nodes():
        file_fwd = str(i) + "_fwd.csv"
        file_gen = str(i) + "_gen.csv"
        file_recv = str(i) + "_recv.csv"
        
        a_fwd = pd.read_csv(p + file_fwd, sep = ",", usecols=['timestamp'])
        a_gen = pd.read_csv(p + file_gen, sep = ",", usecols=['timestamp'])
        a_recv = pd.read_csv(p + file_recv, sep = ",", usecols=['timestamp'])
         
        aux_f1 = a_fwd[a_fwd['timestamp'] > j]
        aux_f1 = aux_f1[aux_f1['timestamp'] < (j + 1)]
        aux_f2 = len(aux_f1)
        
        aux_g1 = a_gen[a_gen['timestamp'] > j]
        aux_g1 = aux_g1[aux_g1['timestamp'] < (j + 1)]
        aux_g2 = len(aux_g1)
        
        aux_r1 = a_recv[a_recv['timestamp'] > j]
        aux_r1 = a_recv[a_recv['timestamp'] < (j + 1)]
        aux_r2 = len(aux_r1)

        traffic_sim.append(aux_f2 + aux_g2 + aux_r2)

In [48]:
signal_sim = pd.DataFrame(np.array(traffic_sim).reshape(n_sim,len(G.nodes())))
signal_sim

,0,1,2,3,4,5,6,7,8,9,...,82,83,84,85,86,87,88,89,90,91
0,215,137,238,14,34,200,15,34,118,93,...,11,13,11,11,12,17,13,11,20,16
1,100,124,128,17,37,99,17,44,99,97,...,14,14,8,14,14,21,11,12,18,20
2,89,123,77,17,41,78,22,34,96,100,...,18,12,14,12,14,24,11,15,18,20
3,85,90,80,16,36,88,20,29,114,109,...,16,12,14,14,16,23,14,15,22,23
4,90,79,86,18,46,91,22,29,103,102,...,20,17,14,20,15,19,14,20,20,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,676,1135,753,679,846,804,790,820,923,1004,...,910,848,638,1016,777,769,921,867,1132,940
716,658,1127,724,682,839,803,793,822,921,1001,...,910,850,636,1010,779,766,916,874,1135,937
717,666,1122,722,684,840,798,794,825,925,999,...,911,850,637,1007,777,766,921,871,1132,935
718,628,1066,694,679,844,739,795,822,864,940,...,913,855,641,1015,775,769,925,871,1134,935


In [49]:
Network = G1.copy() # To exchange the actual network/graph to work with. G1 and G2 in this case.

# GFT-TS based on Matrix Profile

In [50]:
# Creation of series of networks 
T = n_sim
eig_val = []; U = []; Networks = []
for t in range(T):
    peso = signal_sim.loc[t]
    for i in Network.nodes():
        Network.nodes[i]['weight'] = peso[i]
    A = nx.adjacency_matrix(Network, weight='weight')
    Network1 = graphs.Graph(W = A, gtype = 'Outer core')
    eig_val_aux, U_aux = sp.linalg.eigh(Network1.L.toarray())
    eig_val.append(eig_val_aux) # where eig_val[t] is the array of eig_val at time t
    U.append(U_aux) # where U[t] is the array of eigenvec at time t
    Networks.append(Network1)

In [51]:
tau = 1; nlambdas = 10;
spectra = []
for t in range(T):
    peso = signal_sim.loc[t]
    x0 = np.array(peso)
    Networks[t].compute_fourier_basis()
    g = filters.Heat(Networks[t], tau)
    x = g.filter(x0).squeeze()
    x_hat = Networks[t].gft(x).squeeze()
    spectra.extend(np.abs(x_hat).tolist()[1:nlambdas])  # Taking the TOP 10 eigenvectors; out the 1st because it is == 1 ...

In [52]:
window_size = nlambdas # could take other length
mp, mpi = stomp(spectra, window_size)
top_discords = discords(mp, window_size, k=5)

In [53]:
ind_left = [] # len(Network spectrum) = nlambdas; time from 0 up to 719 -- or is it 1 to 720 ? df_outer.shape[0] = 720
for i in range(0, signal_sim.shape[0]):
    if i > 0:
        ind_left.append((nlambdas - 1)*i + 1)
    else: ind_left.append(0)

ind_right = []
for i in range(1, signal_sim.shape[0]+1):
    ind_right.append((nlambdas - 1)*i)

In [54]:
np_array = np.array(spectra)
reshaped_array = np.reshape(np_array, (T, 9))
a_dataframe = pd.DataFrame(reshaped_array)

df_gft = a_dataframe.copy()
for i in top_discords:
    df_gft.drop(bisect(ind_left, i) - 1, axis=0, inplace=True)

mean_gsignal = df_gft.mean(axis = 0)

In [55]:
discords_t = []; motifs_t = []
for i in top_discords:
    discords_t.append(bisect(ind_left, i))
samp_discords = discords_t # number of network with a discord
discords_t = [h*2 for h in discords_t] # minutes from 14:00

discords_find1 = []
for i in samp_discords:
    aux = [abs(x - y) for x,y in zip(list(mean_gsignal),list(spectra[i:(i + 9)]))]
    discords_find1.append(aux)

In [56]:
def influence_point(x, vector): return x > stat.mean(vector) + 1.645*stat.stdev(vector)

In [57]:
output = [idx for idx, element in enumerate(discords_find1[0]) if influence_point(element,discords_find1[0])]
output

[]

# Multiple Matrix Profile - 1 MP per network node 
## (can be in parallel, not now)

In [59]:
# Take first column that is first node TS and run MP... do it for every column of the signal_sim
mean_signal = list(signal_sim.mean(axis=0))
sd_signal = list(signal_sim.std(axis=0))
node_index = 0
window_size2 = 5
candidates = []; p90 = []
for column in  signal_sim: # len(df.columns)
    mp, mpi = stomp(np.array(signal_sim[column]), window_size2)
    top_discords = discords(mp, window_size2, k=2)
    candidates.append(top_discords)
    
    aux90 = mean_signal[node_index] + 1.645*sd_signal[node_index]
    p90.append(aux90)
    node_index = node_index +1

In [60]:
candidates[1][1]

605

In [64]:
anomalies_v = []
cuenta = 0
for i in range(len(candidates)):  # 2 candidates array len = 2 <- {0, 1}
    if (signal_sim[i][candidates[i][0]] - p90[i]) > 0:
        cuenta = cuenta + 1
    if (signal_sim[i][candidates[i][1]] - p90[i]) > 0:
        cuenta = cuenta + 1
cuenta

7

In [62]:
len(candidates) # 2 candidates per column because k = 2 above

92

In [63]:
signal_sim

,0,1,2,3,4,5,6,7,8,9,...,82,83,84,85,86,87,88,89,90,91
0,215,137,238,14,34,200,15,34,118,93,...,11,13,11,11,12,17,13,11,20,16
1,100,124,128,17,37,99,17,44,99,97,...,14,14,8,14,14,21,11,12,18,20
2,89,123,77,17,41,78,22,34,96,100,...,18,12,14,12,14,24,11,15,18,20
3,85,90,80,16,36,88,20,29,114,109,...,16,12,14,14,16,23,14,15,22,23
4,90,79,86,18,46,91,22,29,103,102,...,20,17,14,20,15,19,14,20,20,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,676,1135,753,679,846,804,790,820,923,1004,...,910,848,638,1016,777,769,921,867,1132,940
716,658,1127,724,682,839,803,793,822,921,1001,...,910,850,636,1010,779,766,916,874,1135,937
717,666,1122,722,684,840,798,794,825,925,999,...,911,850,637,1007,777,766,921,871,1132,935
718,628,1066,694,679,844,739,795,822,864,940,...,913,855,641,1015,775,769,925,871,1134,935


In [ ]:
# To create failure / delay in packets management at ~/Desktop/To_Test/new_anx/anx/Components.py
# And run the code above multiple times for several networks
# And save info about TPR = (true positive) / (true positive + false negative)
# TNR = (true negative) / (true negative + false positive)

In [65]:
candidates

[array([287, 126]),
 array([392, 605]),
 array([512, 338]),
 array([448, 379]),
 array([691, 435]),
 array([214, 714]),
 array([ 61, 313]),
 array([671, 307]),
 array([351, 610]),
 array([601, 573]),
 array([249, 286]),
 array([285, 512]),
 array([595, 192]),
 array([108, 622]),
 array([295, 597]),
 array([117, 412]),
 array([441, 654]),
 array([652, 125]),
 array([ 60, 464]),
 array([124,  36]),
 array([254, 392]),
 array([715,  40]),
 array([507, 408]),
 array([439, 362]),
 array([217, 489]),
 array([224, 678]),
 array([713, 686]),
 array([49, 41]),
 array([663, 313]),
 array([460, 674]),
 array([200, 275]),
 array([372, 568]),
 array([428, 249]),
 array([ 64, 713]),
 array([ 98, 496]),
 array([226, 661]),
 array([507,  38]),
 array([699, 542]),
 array([586, 599]),
 array([441, 236]),
 array([646, 595]),
 array([291, 322]),
 array([685, 142]),
 array([148, 292]),
 array([469, 160]),
 array([453,  66]),
 array([ 11, 238]),
 array([ 35, 249]),
 array([409, 370]),
 array([559, 424]),
 a